In [ ]:
# Importing necessary libraries
import os
import pandas as pd
import refinitiv.data as rd

In [ ]:
# Configuration File to communicate with Refinitiv
os.environ["RD_LIB_CONFIG_PATH"] = "../Configuration"

In [ ]:
# Connecting to the Refinitiv API
rd.open_session()

In [ ]:
# Importing the ISINs of all companies in the Refinitiv ESG Universe
df_companies = pd.read_csv("Refinitiv_ESG_Universe.csv", delimiter=";")

In [ ]:
# Retrieving company data that doesn't change with time

# Loop to collect general information for all available companies
comp_data_dict = {}

for isin in df_companies["ISIN"]:
    
    df = rd.get_data(
        universe=isin,
        fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion",
                "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup",
                "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry",
                "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup",
                "TR.SICIndustryGroup", "TR.GICSSector", "TR.SICIndustry",
                "TR.TRBCIndustry", "TR.TRBCIndustryGroup","TR.TRBCBusinessSector",
                "TR.TRBCEconomicSector", "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
                )

    comp_data_dict[isin] = df

# Creating pandas dataframes with the collected data
comp_dfs = []
for ISIN, df in comp_data_dict.items():
    df["key"] = ISIN
    comp_dfs.append(df)

# Concatenating the dataframes into a single one
comp_info_df = pd.concat(comp_dfs)

# Saving the dataframe
comp_info_df.to_csv("comp_info_df.csv")

In [ ]:
# Retrieving company ESG data

# loop to collect ESG data for all available companies
ESG_data_dict = {}

for isin in df_companies["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.TotalReturn", "TR.TRESGScore", "TR.TRESGCScore",
                "TR.TRESGCControversiesScore", "TR.SocialPillarScore",
                "TR.GovernancePillarScore", "TR.EnvironmentPillarScore"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
        )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    ESG_data_dict[isin] = df

# Creating pandas dataframes with the collected data
esg_dfs = []
for ISIN, df in ESG_data_dict.items():
    df["key"] = ISIN
    esg_dfs.append(df)

# Concatenating the dataframes into a single one
comp_tr_esg_df = pd.concat(esg_dfs)

# Saving the new dataframe
comp_tr_esg_df.to_csv("comp_tr_esg_df.csv")

In [ ]:
# Retrieving company financial data

# loop to collect financial data for all available companies
financials_data_dict = {}

for isin in df_companies["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg",
                "TR.EPSMean", "TR.ROAMean", "TR.ROEMean","TR.Volume",
                "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC",
                "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
        )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    financials_data_dict[isin] = df

# Creating pandas dataframes with the collected data
fin_dfs = []
for ISIN, df in financials_data_dict.items():
    df["key"] = ISIN
    fin_dfs.append(df)

# Concatenating the dataframes into a single one
comp_fin_df = pd.concat(fin_dfs)

# Saving the new dataframe
comp_fin_df.to_csv("comp_fin_df.csv")

In [ ]:
rd.close_session()